In [1]:
%%bash
# Careful that this picks up correct version

echo $PATH
gdalinfo --version
INPUT='Copernicus_DSM_COG_10_N39_00_W109_00_DEM.tif' 
OUTPUT='epsg7661_gdal341.tif'

CPL_DEBUG=ON \
PROJ_DEBUG=2 \
gdalwarp -s_srs EPSG:9518 -t_srs EPSG:7661 $INPUT $OUTPUT

/srv/conda/envs/notebook/bin:/srv/conda/condabin:/home/jovyan/.local/bin:/home/jovyan/.local/bin:/srv/conda/envs/notebook/bin:/srv/conda/bin:/srv/npm/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
GDAL 3.4.1, released 2021/12/27


PROJ: pj_open_lib(proj.db): call fopen(/srv/conda/envs/notebook/share/proj/proj.db) - succeeded
GDAL: GDALOpen(../Copernicus_DSM_COG_10_N39_00_W109_00_DEM.tif, this=0x55f93e35dd10) succeeds as GTiff.
GDAL: Using GTiff driver
PROJ: pj_open_lib(proj.ini): call fopen(/srv/conda/envs/notebook/share/proj/proj.ini) - succeeded
GDAL: Computing area of interest: -109, 39.0001, -108, 40.0001
OGRCT: Wrap source at -108.5.
PROJ: pj_open_lib(us_nga_egm08_25.tif): call fopen(/srv/conda/envs/notebook/share/proj/us_nga_egm08_25.tif) - failed
PROJ: pj_open_lib(egm08_25.gtx): call fopen(/srv/conda/envs/notebook/share/proj/egm08_25.gtx) - failed
PROJ: Using https://cdn.proj.org/us_nga_egm08_25.tif
PROJ: pj_open_lib(us_nga_egm08_25.tif): call fopen(/srv/conda/envs/notebook/share/proj/us_nga_egm08_25.tif) - failed
PROJ: pj_open_lib(egm08_25.gtx): call fopen(/srv/conda/envs/notebook/share/proj/egm08_25.gtx) - failed
PROJ: Using https://cdn.proj.org/us_nga_egm08_25.tif
GDAL: GDALDriver::Create(GTiff,epsg766

Creating output file that is 3600P x 3600L.


WARP: Copying metadata from first source to destination dataset
GTiff: ScanDirectories()
GTiff: Opened 1800x1800 overview.
GTiff: Opened 900x900 overview.
GTiff: Opened 450x450 overview.
GDALWARP: Defining SKIP_NOSOURCE=YES
PROJ: Using coordinate operation WGS 84 to EGM2008 height (1)
PROJ: pj_open_lib(us_nga_egm08_25.tif): call fopen(/srv/conda/envs/notebook/share/proj/us_nga_egm08_25.tif) - failed
PROJ: pj_open_lib(egm08_25.gtx): call fopen(/srv/conda/envs/notebook/share/proj/egm08_25.gtx) - failed
PROJ: Using https://cdn.proj.org/us_nga_egm08_25.tif
PROJ: Using coordinate operation WGS 84 to EGM2008 height (1)
PROJ: pj_open_lib(us_nga_egm08_25.tif): call fopen(/srv/conda/envs/notebook/share/proj/us_nga_egm08_25.tif) - failed
PROJ: pj_open_lib(egm08_25.gtx): call fopen(/srv/conda/envs/notebook/share/proj/egm08_25.gtx) - failed
PROJ: Using https://cdn.proj.org/us_nga_egm08_25.tif
GDAL: GDAL_CACHEMAX = 785 MB
GDAL: GDALWarpKernel()::GWKNearestNoMasksOrDstDensityOnlyFloat() Src=0,0,3600

Processing ../Copernicus_DSM_COG_10_N39_00_W109_00_DEM.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.


In [2]:
import rasterio #1.3a, gdal=3.4.1, proj=8.2.1
from rasterio.warp import calculate_default_transform, reproject, Resampling
import numpy as np
import logging
import os

# Necessary to expose PROJ-level logs
os.environ["PROJ_DEBUG"] = "2"

logging.basicConfig(level=logging.DEBUG,
                    handlers=[logging.StreamHandler(),
                              logging.FileHandler('rasterio.log')], 
                    )


#url = 'https://copernicus-dem-30m.s3.eu-central-1.amazonaws.com/Copernicus_DSM_COG_10_N39_00_W109_00_DEM/Copernicus_DSM_COG_10_N39_00_W109_00_DEM.tif'
url = 'Copernicus_DSM_COG_10_N39_00_W109_00_DEM.tif'
with rasterio.Env(GDAL_DISABLE_READDIR_ON_OPEN='EMPTY_DIR',
                   AWS_NO_SIGN_REQUEST='YES',
                   CPL_DEBUG=True,
                   ):
    

    with rasterio.open(url) as src:        
        src_data = src.read()
        print(src_data.mean()) 
        
        # original metadata unaware of vertical reference
        src_crs3D = rasterio.crs.CRS.from_epsg('9518')
        dst_crs = rasterio.crs.CRS.from_epsg('7661')    
        transform, width, height = calculate_default_transform(src_crs3D, 
                                                               dst_crs, 
                                                               src.width, 
                                                               src.height, 
                                                               *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })
        
        with rasterio.open('epsg7661_rio13.tif', 'w', **kwargs) as dst:
            dst_data = np.zeros((1,width,height))
            reproject(source=src_data,
                      destination=dst_data,
                      src_transform=src.transform,
                      src_crs=src_crs3D,
                      dst_transform=transform,
                      dst_crs=dst_crs,
                      resampling=Resampling.nearest,
                      apply_vertical_shift=True,
                     )
            
            # Expect dst_data (ellipsoid) mean to be -15 meters
            print(dst_data.mean())
            dst.write(dst_data)
            
            try:
                np.testing.assert_equal(src_data, dst_data)
                print('Warning: array values unchanged')
            except:
                print('Array values modified!')

DEBUG:rasterio.env:Entering env context: <rasterio.env.Env object at 0x7f4b48192100>
DEBUG:rasterio.env:Starting outermost env
DEBUG:rasterio.env:No GDAL environment exists
DEBUG:rasterio.env:New GDAL environment <rasterio._env.GDALEnv object at 0x7f4b4b79e1f0> created
DEBUG:rasterio._env:GDAL_DATA found in environment.
DEBUG:rasterio._env:PROJ_LIB found in environment.
DEBUG:rasterio._env:Started GDALEnv: self=<rasterio._env.GDALEnv object at 0x7f4b4b79e1f0>.
DEBUG:rasterio.env:Entered env context: <rasterio.env.Env object at 0x7f4b48192100>
DEBUG:rasterio.env:Got a copy of environment <rasterio._env.GDALEnv object at 0x7f4b4b79e1f0> options
DEBUG:rasterio.env:Entering env context: <rasterio.env.Env object at 0x7f4b07ac9700>
DEBUG:rasterio.env:Got a copy of environment <rasterio._env.GDALEnv object at 0x7f4b4b79e1f0> options
DEBUG:rasterio.env:Entered env context: <rasterio.env.Env object at 0x7f4b07ac9700>
DEBUG:rasterio._base:Sharing flag: 0
DEBUG:rasterio._env:CPLE_None in GDAL: GD

2033.3871


DEBUG:rasterio._env:CPLE_None in PROJ: Using https://cdn.proj.org/us_nga_egm08_25.tif
DEBUG:rasterio._env:CPLE_None in PROJ: pj_open_lib(us_nga_egm08_25.tif): call fopen(/srv/conda/envs/notebook/share/proj/us_nga_egm08_25.tif) - failed
DEBUG:rasterio._env:CPLE_None in PROJ: pj_open_lib(egm08_25.gtx): call fopen(/srv/conda/envs/notebook/share/proj/egm08_25.gtx) - failed
DEBUG:rasterio._env:CPLE_None in PROJ: Using https://cdn.proj.org/us_nga_egm08_25.tif
DEBUG:rasterio._env:CPLE_None in GDAL: GDALClose(<VRTDataset rasterYSize="3600" rasterXSize="3600"><VRTRasterBand /><SRS>COMPD_CS["WGS 84 + EGM2008 height",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]],VERT_CS["EGM2008 height",VERT_DATUM["EGM2008 geoid",2005,AUTHORITY["EPSG","1027"]],

2016.963598072295


DEBUG:rasterio._env:CPLE_None in GDAL: GDALClose(epsg7661_rio13.tif, this=0x55b4a4ec3f10)
DEBUG:rasterio._env:CPLE_None in GDAL: GDALClose(../Copernicus_DSM_COG_10_N39_00_W109_00_DEM.tif, this=0x55b4a4e1c300)
DEBUG:rasterio.env:Exiting env context: <rasterio.env.Env object at 0x7f4b48192100>
DEBUG:rasterio.env:Cleared existing <rasterio._env.GDALEnv object at 0x7f4b4b79e1f0> options
DEBUG:rasterio._env:Stopped GDALEnv <rasterio._env.GDALEnv object at 0x7f4b4b79e1f0>.
DEBUG:rasterio.env:Exiting outermost env
DEBUG:rasterio.env:Exited env context: <rasterio.env.Env object at 0x7f4b48192100>


Array values modified!
